In [0]:
# links:
# https://github.com/akshaybhatia10/Reddit-Flair-Detection
# https://github.com/abhishek-parashar/Reddit-flair-detection/blob/master/experiment_log_for_eda.ipynb
# https://github.com/abhishekchopra0907/Reddit-Flair-Detector/blob/master/Jupyter%20Notebooks/data%20analysis.ipynb
# complete all the parts:
==========================

In [0]:
Part II - Exploratory Data Analysis (EDA)
Perform EDA on the data collected in Part I. This is helpful for understanding the data you have
collected. Explain in detail about the analysis you did, intuition behind doing it, output of the
analysis (in terms of graphs or tables), your inference from the output and how it shapes your
future system decisions.

Exploratory Data Analysis (EDA):
---
Exploratory Data Analysis is a process of :-
* understanding the data
* building intuition about the data
* generating and  testing hypothesis about data
* and finding insights in the data


Our Aim:
---
In this part, we will first try to verify assumption of **Part I - Reddit Data Collection**:
* whether feature collected of a post relates to flair of a post
* then, we will try to find insight in the data
* and at last, try to generate new features.

**Note: This part has been divided into three sections:**
1. Univariate Analysis: to explore individual features
2. Bivariate Analysis: to explore feature relations
3. Feature Groups: to explore how can we combine them to get high accuracy

Univariate Analysis:
---

In [0]:
# list of visualization tools to use:
# plt.hist(x)
# plt.plot(x, '.') # x axis row index, y axis feature values
# plt.scatter(range(len(x)), x, c=y)
# df.describe
# x.mean()
# x.var()
# x.value_counts()
# x.isnull()
# plt.scatter(x1, x2)
# pd.scatter_matrix(df)
# df.corr() plt.matshow()
# df.mean().sort_values().plot(style='.')

In [0]:
# exploring anonymized data
# df.dtypes()
# df.info()
# x.value_counts()
# x.isnull()

In [0]:
# dataset cleaning and other things to check
# a feature takes same value for every object in both train and test set.
# traintest.nunique(axis=1) == 1
# train.nunique(axis=1) == 1
# duplicated numerical features
# traintest.T.drop_duplicates()
# for f in categorical_feats:
# traintest[f] = traintest[f].factorize()
# traintest.T.drop_duplicates()

In [0]:
# first find dataset in which we can do eda
# go through notebook and see if 
# they try to do find features

In [0]:
# https://github.com/abhisheksaxena1998/Reddit-Flair-Detector-Application/blob/master/Jupyter%20Notebooks/Part%20II%20-%20Exploratory%20Data%20Analysis%20(EDA).ipynb
# https://github.com/abhisheksaxena1998/Reddit-Flair-Detector/blob/master/reddit_stats.ipynb
# https://github.com/anujarora999/reddit_flare_detection/blob/master/EDA_reddit_flair.ipynb
# https://github.com/azf99/midas-iiitd-internship-challenge-2020/blob/master/Part%202-%20Exploratory%20Data%20Analysis.ipynb
# https://github.com/keshav-b/Reddit-Flair-Detection/blob/master/Models/Part%20II%20-%20Exploratory%20Data%20Analysis%20(EDA).ipynb
# https://github.com/Mihir-Rajora20/Reddit-Flair-Detector/blob/master/Data%20Exploration%20and%20Modification.ipynb
# https://github.com/Mihir-Rajora20/Reddit-Flair-Detector/blob/master/Data%20Extraction%20Using%20Pushshifts.ipynb
# https://github.com/sawarn69/Reddit-Flair-Detection/blob/master/Download%20data%2B%20EDA.ipynb
# https://github.com/Shaurya-L/Reddit-Flair-Detector/blob/master/Data%20Analysis.ipynb
# https://nbviewer.jupyter.org/github/someshsingh22/FlaiReddit-MIDAS/blob/master/Part-2-EDA.ipynb
# https://github.com/yatharthmathur/RedditAnalysis/blob/master/EDA/EDA.ipynb
# https://github.com/yatharthmathur/RedditAnalysis/blob/master/EDA/Visualization.ipynb

# use this repo
# https://github.com/akshaybhatia10/Reddit-Flair-Detection
# https://github.com/harshita219/Reddit-Flair-Detection
# https://github.com/Himanshu-Garg/Reddit-flair-detection
# https://github.com/Mihir-Rajora20/Reddit-Flair-Detector # for readme
# important ones
# https://github.com/manya16400/Reddit-Flair-Detection/blob/master/Notebooks/1.Data%20Scraping.ipynb
# https://github.com/kshitijgulati98/reddit-flair-detector/blob/master/Jupyter%20Notebooks/Part%202%20-%20EDA.ipynb
# https://github.com/Himanshu-Garg/Reddit-flair-detection/blob/master/other/1year_data.ipynb
# https://github.com/harshita219/Reddit-Flair-Detection/blob/master/EDA.ipynb
# https://github.com/harshita219/Reddit-Flair-Detection/blob/master/extract_data.ipynb
# https://github.com/akshaybhatia10/Reddit-Flair-Detection/blob/master/notebooks/data_aquisition.ipynb
# https://github.com/akshaybhatia10/Reddit-Flair-Detection/blob/master/app/app.py
# https://github.com/BhavyaC16/FlairifyMe/blob/master/app.py
# https://github.com/parasmehan123/RedFlair/blob/master/model.py


In [0]:
from google.colab import files

files.upload()

In [0]:
import pandas as pd

data = pd.read_csv('data.csv')
data.head()

In [0]:
data.selftext

In [0]:
data.columns

In [0]:
data.isnull().sum()

In [0]:
data['created_utc'] = pd.to_datetime(data['created_utc'], unit='s')
data

In [0]:
data.dtypes

In [0]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (15, 5)
data.hist();

In [0]:
data.created_utc.value_counts().sort_values()

In [0]:
plt.hist(data.created_utc, bins=30);

In [0]:
data.num_comments.value_counts().sort_values()

In [0]:
plt.hist(data.num_comments, bins=20)

In [0]:
import numpy as np

hist, bin_edges = np.histogram(data.num_comments)

In [0]:
hist

In [0]:
bin_edges[:-1]

In [0]:
min(bin_edges)

In [0]:
max(bin_edges)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
plt.bar(bin_edges[:-1], hist, width = 0.5, color='#0504aa',alpha=0.7)
plt.xlim(min(bin_edges), max(bin_edges))

In [0]:
plt.figure(figsize=[10,8])

plt.bar(bin_edges[:-1], hist, width = 0.5, color='#0504aa',alpha=0.7)
plt.xlim(min(bin_edges), max(bin_edges))
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value',fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('Frequency',fontsize=15)
plt.title('Normal Distribution Histogram',fontsize=15)
plt.show()

In [0]:
np.random.seed(100)
np_hist = np.random.normal(loc=0, scale=1, size=1000)

In [0]:
hist,bin_edges = np.histogram(np_hist)

In [0]:
bin_edges[:-1]

In [0]:
import matplotlib.pyplot as plt
# to modify 
plt.rcParams['figure.figsize'] = (15, 5)
data.link_flair_text.hist(bins=20);

In [0]:
# making sure data is intuitive: 1000 observations and 11 features
data.shape

In [0]:
import matplotlib.pyplot as plt
# to modify 
plt.rcParams['figure.figsize'] = (15, 5)
data.link_flair_text.hist(bins=20);

In [0]:
data.columns

In [0]:
plt.scatter(data.author, data.created_utc, c=data.link_flair_text.factorize()[0]);

In [0]:
# encode text to numeric

In [0]:
pd.plotting.scatter_matrix(data)

In [0]:
data.corr()

In [0]:
plt.matshow()

In [0]:
data.mean().sort_values().plot(style='.');

In [0]:
data.

In [0]:
plt.scatter(data.created_utc, data.score, c=data.link_flair_text.factorize()[0]);

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[5,5])
plt.plot(data.over_18, '.');

In [0]:
# x-axis row index, y-axis feature value
# plot index versus value
plt.figure(figsize=[5,5])
plt.scatter(range(len(data.over_18)), data.over_18, c=data.link_flair_text.factorize()[0]);

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[5,5])
plt.plot(data.score, '.');

In [0]:
# x-axis row index, y-axis feature value
# plot index versus value
plt.figure(figsize=[5,5])
plt.scatter(range(len(data.score)), data.score, c=data.link_flair_text.factorize()[0]);

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[5,5])
plt.plot(data.num_comments, '.');

In [0]:
# x-axis row index, y-axis feature value
# plot index versus value
plt.figure(figsize=[5,5])
plt.scatter(range(len(data.num_comments)), data.num_comments, c=data.link_flair_text.factorize()[0]);

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[5,5])
plt.plot(data.link_flair_text, '.');

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[5,5])
plt.plot(data.created_utc, '.');

In [0]:
# x-axis row index, y-axis feature value
# plot index versus value
plt.figure(figsize=[5,5])
plt.scatter(range(len(data.created_utc)), data.created_utc, c=data.link_flair_text.factorize()[0]);

In [0]:
# x-axis row index, y-axis feature value
plt.figure(figsize=[30,30])
plt.plot(data.author, '.');

In [0]:
# x-axis row index, y-axis feature value
# plot index versus value
plt.figure(figsize=[30,30])
plt.scatter(range(len(data.author)), data.author, c=data.link_flair_text.factorize()[0]);

In [0]:
# feature extraction and preprocessing with respect to models
# bag of words
# embeddings (word to vector)
# tfidf
# N-grams
# lowercase
# lemmatization
# stemming
# stopwords
# word2vec glove, fastText, doc2vec

In [0]:
data

Exploring individual features:
---
We will first explore individual features and then we will try to explore their relations

In [0]:
data.nunique().sort_values()

In [0]:
data.selftext.value_counts().sort_values()

In [0]:
!wget https://raw.githubusercontent.com/abhishekchopra0907/Reddit-Flair-Detector/master/reddit-india-data.csv

In [0]:
#downloading libraries
import pandas as pd 
import numpy as np
import matplotlib as plt
import pandas as pd
import nltk
import re
import numpy as np
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt
from collections import Counter
import pickle

#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

In [0]:
from keras.layers import Dense, Dropout, Flatten  
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.layers import Dense, Conv2D
from keras.layers import Flatten
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.models import Sequential
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

In [0]:
data=pd.read_csv('reddit-india-data.csv')
data=data.fillna('NO VALUE')
feature_combine = data["title"] + data["comments"]
data = data.assign(feature_combine = feature_combine)
data

In [0]:
def text_extractor(text,text_type):
    title_list=[]
    for i in range(len(text)):
        title_list.append(text[text_type][i])
    return title_list

def preProcessData(dataa):
    stopwords_en = list(set(stopwords.words('english')))
    def split(word): 
        return [char for char in word]   
    punchList = split(punctuation)

    print(stopwords_en)
    print('Punctuation :', punchList)

    wordTokenList = [word_tokenize(sent) for sent in dataa]
    lowercasingList = [[word.lower() for word in sentence] for sentence in wordTokenList]
    noStopWordList = [[word for word in sentence if word not in stopwords_en] for sentence in lowercasingList]
    noPunchList = [[re.sub(r'([^\s\w]|_)+', '', word) for word in sentence] for sentence in noStopWordList]
    #noPunchList = [[word for word in sentence if word not in punchList] for sentence in noStopWordList]
    PP_data = [[word for word in sentence if word] for sentence in noPunchList]
    return PP_data

def cleaner(PP_data):
    PP_corr=[]
    for title in PP_data:
        temp=[]
        for word in title:
            if word in embeddings_index.keys():
                    temp.append(word)
        PP_corr.append(temp)
    return PP_corr

def pad_features(titles,max_len):
    features=[]
    for title in titles:
        temp=[]
        for i in range(max_len):
            if i < len(title):
                temp.append(title[i])
            else:
                temp.append("ignore_nan")
        features.append(temp)
    return features


def vectorizer(summ_data,embeddings_index):
    summ_vec=[]
    for list_w in summ_data:
        temp=[]
        for word in list_w:
            if word in embeddings_index.keys():
                temp.append(embeddings_index[word])
        summ_vec.append(temp)
    return summ_vec

def sent_vectorizer(vector):
    sent_vect=[]
    for sent in vector:
        n=0
        temp=np.zeros(200)
        for word in sent:
            temp=temp+word
            n=n+1
        sent_vect.append(temp/n)
    return sent_vect
'''
import os
glove_dir = 'glove'
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
z=np.zeros(200)
embeddings_index['ignore_nan']=z
'''
def reshaper(t_x):
    x = np.reshape(t_x, (t_x.shape[0], 1,t_x.shape[1]))
    return x

def data_to_ready(data,embed):
    t_list=text_extractor(data,'title')
    pp_data=preProcessData(t_list)
    clean=cleaner(pp_data)
#     max_len=max_finder(clean)
    clean=pad_features(clean,256)
    final=vectorizer(clean,embed)
    final=sent_vectorizer(final)
    final=np.array(final)
#     re_final=reshaper(final)
#     return re_final
    return final

In [0]:
def shuffler(data):
    df_sh=shuffle(data)
    df_sh=df_sh.reset_index()
    return df_sh
def one_shotter(z,colt):
    y=z[colt]
    y=np.array(y)
    y=pd.get_dummies(y)
    return y

In [0]:
sh = shuffler(data)
sh

In [0]:
import shuffle

In [0]:
!wget https://raw.githubusercontent.com/akshala/reddit-flair-detector/master/initial_final.csv

In [0]:
import nltk
nltk.download('all')
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean(text):
	text = BeautifulSoup(text, "lxml").text # HTML decoding
	text = text.lower() # lowercase text
	text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
	text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
	text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
	return text

In [0]:
import pandas

data = pandas.read_csv('initial_final.csv')

data

In [0]:
data[['title']] = data.apply(lambda x: clean(x['title']), axis=1)
data

In [0]:
data[['comments']] = data.apply(lambda x: clean(x['comments']), axis=1)
data

In [0]:
y = data['link_flair_text']
y

In [0]:
flairs = list(set(y))

flairs

In [0]:
X = data['title'] + data['url'] + data['comments'] + data['author'].fillna('') + data['id']
X

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
X_train

In [0]:
import numpy
import pandas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
import pickle

In [0]:
logreg = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression(n_jobs=1, C=1e5)),])
logreg.fit(X_train, y_train)

In [0]:
y_pred = logreg.predict(X_test)
print('accuracy (combined)', accuracy_score(y_pred, y_test))

Bag-of-Words:
---
The bag-of-words model is a way of representing text data when modeling text with machine learning algorithms.

In [0]:
# basic preprocessing:

import nltk
def to_lower(text):
  """
  Converting text to lower case as in converting "HELLO" to "hello"
  """
  return ' '.join([w.lower() for w in word_tokenize(text)])

In [0]:
feature extraction from text
bag of words
embeddings word 2 vec
coutvectorizer sklearn
term frequency
inverse document 
text.tfidfvectorizer sklearn
n-grams 
sklearn ngram_range, analyzer
text preprocessing
lowercase
countvectorizer from sklearn does this by default
lemmatization advanced preprocessing
stemming
stopwords
sklearn.feature_extraction.text.countvectorizer
max_df

Stemming:
---
the process of removing prefixes and suffixes from words until we are left with its stem, which carries most of its meaning. So cars is stemmed to car

In [0]:
import nltk

nltk.download()

In [0]:
# different stemmers available in different languages in Python nltk
# for the English language, you can choose between PorterStammer or LancasterStammer
# 

In [0]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [0]:
# create an object of class PorterStemmer
porter = PorterStemmer()
lancaster = LancasterStemmer()
# provide a word to be stemmed
print("Porter Stemmer")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))
print("Lancaster Stemmer")
print(lancaster.stem("cats"))
print(lancaster.stem("trouble"))
print(lancaster.stem("troubling"))
print(lancaster.stem("troubled"))

In [0]:
# PorterStemmer use Suffix Stripping to produce stems
# PorterStemmer algorithm does not follow linguistics rather a set of 05 rules
# for different cases that are applied in phases (step by step) to generate
# stems

In [0]:
# A list of words to be stemmed
word_list = ["friend", "friendship", "friends", "stabil", "destabilize", "misunderstanding", "railroad", "moonlight", "football"]
print("{0:20}{1:20}{2:20}".format("Word", "Porter Stemmer", "lancaster Stemmer"))
for word in word_list:
  print("{0:20}{1:20}{2:20}".format(word, porter.stem(word), lancaster.stem(word)))

In [0]:
# The LancasterStemmer (Paice-Husk stemmer) is an iterative algorithm with
# rules saved externally. One table containing about 120 rules indexed by the
# last letter of a suffix. On each iteration, it tries to find an applicable
# rule by the last character of the word.

In [0]:
sentence = "Pythoners are very intelligent and work very pythonly and now they are pythoning their way to success."
porter.stem(sentence)

In [0]:
# stemmers sees the entire sentence as a word, so it returns it as it is.
# We need to stem each word in the sentence and return a combined sentence.
# to separate the sentence into words, you can use tokenizer.

In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize

def stemSentence(sentence):
  token_words = word_tokenize(sentence)
  token_words
  stem_sentence = []
  for word in token_words:
    stem_sentence.append(porter.stem(word))
    stem_sentence.append(" ")
  return "".join(stem_sentence)

x = stemSentence(sentence)
print(x)

In [0]:
nltk.download()

In [0]:
nltk.corpus.gutenberg.fileids()

In [0]:
text_file = nltk.corpus.gutenberg.words('melville-moby_dick.txt')
my_lines_list = []
for line in text_file:
  my_lines_list.append(line)

my_lines_list

In [0]:
text_file = nltk.corpus.gutenberg.words('melville-moby_dick.txt')
text_file

In [0]:
my_lines_list

In [0]:
# SnowballStemmer as a language to create to non-English stemmers. One
# can program one's own language stemmer using snowball.

In [0]:
from nltk.stem.snowball import SnowballStemmer

englishStemmer = SnowballStemmer("english")
englishStemmer.stem("having")

In [0]:
nltk.download()

In [0]:
englishStemmer2 = SnowballStemmer("english", ignore_stopwords=True)
englishStemmer2.stem("having")

In [0]:
spanishStemmer = SnowballStemmer("spanish", ignore_stopwords=True)
spanishStemmer.stem("Corriendo")

In [0]:
# Lemmization reduces the inflected words properly ensuring that the root 
# word belongs to the language. A lemma (plural lemmas or lemmata) is the 
# cononical form, dictionary form, or citatino form of a set of words.

In [0]:
nltk.download()

In [0]:
import nltk

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

sentence = "He was running and eating at same time. He has bad habit of swimming after playing long hours in the Sun."
punctuations = "?:!.,;"
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
  if word in punctuations:
    sentence_words.remove(word)


sentence_words

In [0]:
print("{0:20}{1:20}".format("Word", "Lemma"))
for word in sentence_words:
  print("{0:20}{1:20}".format(word, wordnet_lemmatizer.lemmatize(word)))

In [0]:
# you need to provide the context in which you want to lemmatize that is
# the parts-of-speech (POS). This is done by giving the value for pos
# parameter in wordnet_lemmatizer.lemmatize

In [0]:
for word in sentence_words:
  print("{0:20}{1:20}".format(word, wordnet_lemmatizer.lemmatize(word, pos="v")))